<a href="https://colab.research.google.com/github/mostafadentist/python-ipynb/blob/main/The_Simplex_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

# Initial simplex tableau for the above problem
tableau = np.array([
    [2, 1, 1, 0, 100],  # constraint 1
    [1, 3, 0, 1, 90],   # constraint 2
    [-3, -5, 0, 0, 0]   # objective row
], dtype=float)

df = pd.DataFrame(tableau,
                  columns=["x1", "x2", "s1", "s2", "RHS"])
print(df)

    x1   x2   s1   s2    RHS
0  2.0  1.0  1.0  0.0  100.0
1  1.0  3.0  0.0  1.0   90.0
2 -3.0 -5.0  0.0  0.0    0.0


In [2]:
def pivot(tableau, row, col):
    tableau[row] /= tableau[row, col]
    for r in range(len(tableau)):
        if r != row:
            tableau[r] -= tableau[r, col] * tableau[row]
    return tableau

In [3]:
def simplex(tableau):
    while np.any(tableau[-1,:-1] < 0):  # while there are negative coefficients in objective row
        col = np.argmin(tableau[-1,:-1])  # entering variable
        ratios = []
        for i in range(len(tableau)-1):
            if tableau[i,col] > 0:
                ratios.append(tableau[i,-1] / tableau[i,col])
            else:
                ratios.append(np.inf)
        row = np.argmin(ratios)  # leaving variable
        tableau = pivot(tableau, row, col)
    return tableau

final_tableau = simplex(tableau.copy())
df_final = pd.DataFrame(final_tableau,
                        columns=["x1", "x2", "s1", "s2", "RHS"])
print("Final Tableau:\n", df_final)

Final Tableau:
     x1   x2   s1   s2    RHS
0  1.0  0.0  0.6 -0.2   42.0
1  0.0  1.0 -0.2  0.4   16.0
2  0.0  0.0  0.8  1.4  206.0


In [4]:
solution = {"x1": 0, "x2": 0}
for i, var in enumerate(["x1","x2","s1","s2"]):
    col = final_tableau[:,i]
    if list(col).count(1) == 1 and list(col).count(0) == len(col)-1:
        row = np.where(col == 1)[0][0]
        solution[var] = final_tableau[row,-1]

Z = final_tableau[-1,-1]
print("Optimal Solution:", solution)
print("Optimal Value Z =", Z)

Optimal Solution: {'x1': np.float64(42.0), 'x2': np.float64(16.0)}
Optimal Value Z = 206.0


In [5]:
from scipy.optimize import linprog

c = [-3, -5]  # maximize 3x1+5x2 → minimize -Z
A = [[2,1],[1,3]]
b = [100,90]

res = linprog(c, A_ub=A, b_ub=b, bounds=(0,None))
print("SciPy Optimal Solution:", res.x)
print("SciPy Optimal Z =", -res.fun)

SciPy Optimal Solution: [42. 16.]
SciPy Optimal Z = 206.0
